In [70]:
import numpy as np
import pandas as pd

GRID = 'grid'
MG = 'mg'
SHS = 'shs'
ELECTRIFICATION_OPTIONS = [GRID, MG, SHS]
BAU_SCENARIO = 'bau'
SE4ALL_SCENARIO = 'se4all'
SE4ALL_FLEX_SCENARIO = 'se4all_shift'
PROG_SCENARIO = 'prog'
SCENARIOS = [BAU_SCENARIO, SE4ALL_SCENARIO, PROG_SCENARIO]

# Names for display
SCENARIOS_DICT = {
    BAU_SCENARIO: 'BaU',
    SE4ALL_SCENARIO: 'SE4All',
    PROG_SCENARIO: 'prOG',
}

ELECTRIFICATION_DICT = {
    GRID: 'Grid',
    MG: 'Mini Grid',
    SHS: 'Solar Home System'
}

# column names of the exogenous results
POP_GET = ['pop_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_GET = ['hh_get_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_CAP = ['hh_%s_capacity' % opt for opt in ELECTRIFICATION_OPTIONS]
HH_SCN2 = ['hh_cap_scn2_%s_capacity' % opt for opt in ELECTRIFICATION_OPTIONS]
INVEST = ['%s_investment_cost' % opt for opt in [MG, SHS]]
INVEST_CAP = ['tier_capped_%s_investment_cost' % opt for opt in [MG, SHS]]
GHG = ['ghg_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
GHG_CAP = ['tier_capped_ghg_%s_2030' % opt for opt in ELECTRIFICATION_OPTIONS]
EXO_RESULTS = POP_GET + HH_GET + HH_CAP + HH_SCN2 + INVEST + INVEST_CAP + GHG + GHG_CAP

# source http://www.worldbank.org/content/dam/Worldbank/Topics/Energy%20and%20Extract/
# Beyond_Connections_Energy_Access_Redefined_Exec_ESMAP_2015.pdf
MIN_TIER_LEVEL = 3
MIN_RATED_CAPACITY = {1: 3, 2: 50, 3: 200, 4: 800, 5: 2000}  # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {1: 4.5, 2: 73, 3: 365, 4: 1250, 5: 3000}  # index is TIER level [kWh/a]
RATIO_CAP_CONSUMPTION = {}

# Investment Cost Source: Arranz and Worldbank,
# BENCHMARKING STUDY OF SOLAR PV MINIGRIDS INVESTMENT COSTS, 2017 (Jabref)
# unit is USD per household
MEDIAN_INVESTMENT_COST = {1: 742, 2: 1273, 3: 2516, 4: 5277, 5: 5492}

# drives for the socio-economic model
IMPACT_FACTORS = pd.DataFrame(
    {
        MG: [3, 13. / 6, 19. / 6, 3.25, 11. / 3],
        SHS: [23. / 12, 4.5, 37. / 12, 17. / 6, 41. / 12],
        'labels': [
            'high_gdp',
            'high_mobile_money',
            'high_ease_doing_business',
            'low_corruption',
            'high_grid_weakness'
        ]
    }
)
IMPACT_FACTORS = IMPACT_FACTORS.set_index('labels')

MENTI_DRIVES = ['gdp', 'mobile_money', 'ease_doing_business', 'corruption', 'weak_grid']

# $RT_shift_factors.$P$2
WEIGHT_MENTIS = 0.2
# -->WEIGHT_GRID = 0.8 ($RT_shift_factors.$O$2)  and  WEIGHT_GRID = 1 - WEIGHT_MENTIS
RISE_INDICES = ['rise_%s' % opt for opt in ELECTRIFICATION_OPTIONS]
SHIFT_MENTI = ['shift_menti_mg', 'shift_menti_shs']

BASIC_ROWS = [
    'People share',
    'People (k)',
    'HH (k)',
    'HH cap. (MW)',
    'HH cap. (MW) (TIER + 1)',
    'Investment MUSD',
    'Investment (TIER + 1) MUSD',
]
# labels of the columns of the result tables
LABEL_COLUMNS = ELECTRIFICATION_DICT.copy()
# a column for the row labels
LABEL_COLUMNS['labels'] = ''
LABEL_COLUMNS['total'] = 'Total'
BASIC_COLUMNS_ID = ['labels'] + ELECTRIFICATION_OPTIONS + ['total']
GHG_COLUMNS_ID = ['labels'] + ELECTRIFICATION_OPTIONS + ['total']
COMPARE_COLUMNS_ID = ['labels']
for opt in ELECTRIFICATION_OPTIONS + ['total']:
    COMPARE_COLUMNS_ID.append(opt)
    COMPARE_COLUMNS_ID.append('comp_{}'.format(opt))


## Test TIER level attribution

In [ ]:
from data_preparation import _find_tier_level

for hh_cons in np.array([1, 50, 100, 400, 2000, 5000]):
    print(_find_tier_level(hh_cons, 2))

In [ ]:
from data_preparation import _slope_capacity_vs_yearly_consumption
RATIO_CAP_CONSUMPTION = {}
TIER_LEVELS = [1, 2, 3, 4, 5]
for tier_lvl in [1, 2, 3, 4]:
    RATIO_CAP_CONSUMPTION[tier_lvl] = _slope_capacity_vs_yearly_consumption(tier_lvl)
RATIO_CAP_CONSUMPTION

In [ ]:
MIN_RATED_CAPACITY = {1: 3, 2: 50, 3: 200, 4: 800, 5: 2000}  # index is TIER level [W]
MIN_ANNUAL_CONSUMPTION = {1: 4.5, 2: 73, 3: 365, 4: 1250, 5: 3000}  # index is TIER level [kWh/a]

In [ ]:
for tier_lvl in TIER_LEVELS:
    print(MIN_RATED_CAPACITY[tier_lvl] / MIN_ANNUAL_CONSUMPTION[tier_lvl])

In [ ]:
cap = [MIN_RATED_CAPACITY[i] for i in TIER_LEVELS]
df = pd.DataFrame(cap)
df[0].plot()

In [ ]:
cons = [MIN_ANNUAL_CONSUMPTION[i] for i in TIER_LEVELS]
df = pd.DataFrame(cons)
df[0].plot()

In [ ]:
from data_preparation import IMPACT_FACTORS
IMPACT_FACTORS.index.to_list()

In [ ]:
for opt in [MG, SHS]:
    for input_name in IMPACT_FACTORS.index.to_list():
        #print("Input('impact-{}-{}-input', 'value'),".format(opt, input_name.replace('_', '-')))
        print('impact_{}_{},'.format(opt, input_name.replace('high', '')))



In [ ]:
country,country_iso, ghg_grid_2030,ghg_mg_2030,ghg_shs_2030,ghg_no_access_2030,tier_capped_ghg_grid_2030,tier_capped_ghg_mg_2030,tier_capped_ghg_shs_2030,tier_capped_ghg_no_access_2030

# Tests of the model

In [ ]:
from data.data_preparation import compute_ndc_results_from_raw_data
SCENARIOS_DATA = {
    sce: compute_ndc_results_from_raw_data(sce, MIN_TIER_LEVEL).to_json() for sce in SCENARIOS
}

In [ ]:
xls_bau = pd.read_csv('data/xls_bau.csv', float_precision='high')
xls_se = pd.read_csv('data/xls_se.csv', float_precision='high')
xls_prog = pd.read_csv('data/xls_prog.csv', float_precision='high')

invest_bau = pd.read_csv('data/invest_bau.csv', float_precision='high')
invest_se = pd.read_csv('data/invest_se.csv', float_precision='high')
invest_prog = pd.read_csv('data/invest_prog.csv', float_precision='high')

## Test Investment cost

### BaU

In [71]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_bau = pd.read_csv('data/ghg_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

problems with  ghg_grid_2030 ['IDN']
1
problems with  ghg_mg_2030 ['IDN']
1
problems with  tier_capped_ghg_grid_2030 ['IDN']
1
problems with  tier_capped_ghg_mg_2030 ['IDN']
1


,ghg_grid_2030,ghg_mg_2030,ghg_shs_2030,tier_capped_ghg_grid_2030,tier_capped_ghg_mg_2030,tier_capped_ghg_shs_2030
country_iso,,,,,,
IDN,1885.016081,290.925261,0,2874.960978,443.709091,0


### SE4All

In [72]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_se = pd.read_csv('data/ghg_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

problems with  ghg_grid_2030 ['IDN', 'ZWE']
2
problems with  ghg_mg_2030 ['IDN', 'ZWE']
2
problems with  tier_capped_ghg_grid_2030 ['IDN', 'ZWE']
2
problems with  tier_capped_ghg_mg_2030 ['IDN', 'ZWE']
2
problems with  tier_capped_ghg_shs_2030 ['ZWE']
1


,ghg_grid_2030,ghg_mg_2030,ghg_shs_2030,tier_capped_ghg_grid_2030,tier_capped_ghg_mg_2030,tier_capped_ghg_shs_2030
country_iso,,,,,,
ZWE,533773.426236,-54465.429772,0,-61033.637465,-148762.394563,452429.512592
IDN,701.826870,43.389945,0,1070.401937,66.176836,0.000000


### prOG

In [73]:
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
ghg_prog = pd.read_csv('data/ghg_prog.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = GHG + GHG_CAP

df_diff = ghg_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

problems with  ghg_grid_2030 ['BWA', 'IDN']
2
problems with  ghg_mg_2030 ['BWA', 'IDN']
2
problems with  tier_capped_ghg_grid_2030 ['BWA', 'IDN']
2
problems with  tier_capped_ghg_mg_2030 ['BWA', 'IDN']
2


,ghg_grid_2030,ghg_mg_2030,ghg_shs_2030,tier_capped_ghg_grid_2030,tier_capped_ghg_mg_2030,tier_capped_ghg_shs_2030
country_iso,,,,,,
IDN,100.252388,106.454264,0.0,152.901455,162.360342,0.0
BWA,116381.377450,-5866.736103,0.0,140613.826964,-7088.283653,0.0


## Test Investment cost

### BaU

In [74]:
bau_df = pd.read_json(SCENARIOS_DATA[BAU_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_bau = pd.read_csv('data/invest_bau.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_bau[COMP_COLS] - bau_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

problems with  mg_investment_cost ['IDN', 'IND']
2
problems with  shs_investment_cost ['IDN']
1
problems with  tier_capped_mg_investment_cost ['IDN', 'IND', 'NAM']
3
problems with  tier_capped_shs_investment_cost ['IDN']
1


,mg_investment_cost,shs_investment_cost,tier_capped_mg_investment_cost,tier_capped_shs_investment_cost
country_iso,,,,
IND,-6.705028e+01,16.193810,-1.617987e+02,32.387642
NAM,6.181916e+00,-0.037684,6.102219e+01,-0.083999
IDN,3.119468e+06,359668.045306,4.884199e+06,719336.090611


### SE4All

In [75]:
se_df = pd.read_json(SCENARIOS_DATA[SE4ALL_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_se = pd.read_csv('data/invest_se.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_se[COMP_COLS] - se_df[COMP_COLS]

def highlight_mismatch(col, eps=20):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

problems with  mg_investment_cost ['IDN', 'NAM']
2
problems with  shs_investment_cost ['IDN']
1
problems with  tier_capped_mg_investment_cost ['IDN', 'NAM']
2
problems with  tier_capped_shs_investment_cost ['IDN']
1


,mg_investment_cost,shs_investment_cost,tier_capped_mg_investment_cost,tier_capped_shs_investment_cost
country_iso,,,,
NAM,23.645296,4.470348e-08,236.900143,2.682209e-07
IDN,465251.938758,1.261713e+06,728452.082689,2.523425e+06


### prOG

In [76]:
prog_df = pd.read_json(SCENARIOS_DATA[PROG_SCENARIO]).set_index('country_iso').sort_index(ascending=True)
invest_prog = pd.read_csv('data/invest_prog.csv', float_precision='high').set_index('country_iso').sort_index(ascending=True)

COMP_COLS = INVEST + INVEST_CAP

df_diff = invest_prog[COMP_COLS] - prog_df[COMP_COLS]

def highlight_mismatch(col, eps=40):
    return df_diff.loc[np.abs(df_diff[col]) > eps]

l = []
for col in COMP_COLS:
    temp = highlight_mismatch(col).index.to_list()
    if temp:
        print('problems with ', col, temp)
        print(len(temp))
    l = l + temp
len(set(l))
l = list(set(l))
df_diff.loc[l]

problems with  mg_investment_cost ['BWA', 'IDN']
2
problems with  shs_investment_cost ['BWA', 'IDN']
2
problems with  tier_capped_mg_investment_cost ['BWA', 'IDN', 'NAM']
3
problems with  tier_capped_shs_investment_cost ['BWA', 'IDN']
2


,mg_investment_cost,shs_investment_cost,tier_capped_mg_investment_cost,tier_capped_shs_investment_cost
country_iso,,,,
NAM,2.481585e+01,-7.450581e-08,2.486279e+02,2.980232e-07
IDN,1.141464e+06,1.376495e+06,1.787207e+06,2.752989e+06
BWA,-5.215774e+07,-7.589804e+06,-6.395015e+07,-1.691781e+07
